In the same directory, there is `talents.txt` file. That is PDF content provided by Gallup(tm) please note that they are owner of this content.

Countries taken from here: https://gist.github.com/pamelafox/986163 - I had to do some changes, that is why I have a copy.

To run this script pandas, folium, xlrd, xlwt modules are needed.

Have a fun!

Take `talents.txt`, and creates `talents2.xls` with geo coordinates. 

In [1]:
from helper import convert_to_excel

# convert_to_excel('talents.txt', 'talents2.xls')

Read dataframe from `talents2.xls`

In [2]:
import pandas as pd

df = pd.read_excel('talents2.xls')

remove females, males, all entries (the ones without coordinates)

In [3]:
df = df.dropna(subset=['lon'])
df = df.dropna(subset=['lat'])

Print out map with Learners!

In [10]:
import folium
from folium import plugins

def put_piechart(folium_map, location, row):
    popup_text = f'{index}<br> country: {row["country"]}<br> respondants: {row["respondants"]}'
    radius = float(row['Learner'])/1000
    if radius > 60:
        radius = 60
    folium.CircleMarker(location=(row["lat"], row["lon"]),
                        radius=radius,
                        popup=popup_text,
                        color="#007849",
                        fill=True,
                        fill_color="#015627",
                        fill_opacity=0.7
                       ).add_to(folium_map)
    #help(plugins.SemiCircle)
    #plugins.SemiCircle((45, 3), radius=400000, start_angle=50, stop_angle=200,
    #               color='green', fill_color='green', opacity=0,
    #               popup='start angle - 50 degrees, stop angle - 200 degrees'
    #              ).add_to(folium_map)    


folium_map = folium.Map(location=(52.2297700, 21.0117800), zoom_start=1)

for index, row in df.iterrows():                    
    put_piechart(folium_map, location=(row["lat"], row["lon"]), row=row)

Sample output stored in html - github is not supporting folium....

In [11]:
#folium_map.save("sample_learners.html")

Print out output on screen.

In [12]:
folium_map